In [7]:
import pandas as pd

df_yelp = pd.read_csv('./sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df_amazon = pd.read_csv('./sentiment labelled sentences/amazon_cells_labelled.txt', names=['sentence', 'label'], sep='\t')
df_imdb = pd.read_csv('./sentiment labelled sentences/imdb_labelled.txt', names=['sentence', 'label'], sep='\t')

print("Yelp: ", df_yelp)
print("Amazon: ", df_amazon)
print("IMDB: ", df_imdb)

Yelp:                                                sentence  label
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]
Amazon:                                                sentence  label
0    So there is no way for me to plug it in here i...      0
1                          Good case, Excellent value.      1
2                            